# Case: Emneord

**TODO**
- text

In [1]:
import bibdata
from collections import Counter
import numpy

In [2]:
bibdata.meta[1234]

{'_id': '870970-basis:51363035',
 'creator': 'Jakob Martin Strid',
 'idx': 1234,
 'language': 'Dansk',
 'subject-term': ['årstider',
  'for 5 år',
  'sk',
  'sjove bøger',
  'Skønlitteratur',
  'for 4 år',
  'for 6 år',
  'for 3 år',
  'venner',
  'vejret',
  'dyrefortællinger'],
 'title': 'Mimbo Jimbo og den lange vinter',
 'type': 'book'}

In [3]:
subjects = []
for bib in bibdata.meta:
    for subject in bib.get('subject-term', []):
        subjects.append(subject)
Counter(subjects).most_common(10)

[('sk', 7036),
 ('Skønlitteratur', 6445),
 ('for 6 år', 1109),
 ('krimi', 1100),
 ('for 5 år', 1097),
 ('for 7 år', 1063),
 ('let at læse', 1047),
 ('for 4 år', 999),
 ('for 8 år', 932),
 ('Spillefilm', 904)]

In [4]:
"; ".join([subject for (subject, count) in Counter(subjects).most_common(150)])

'sk; Skønlitteratur; for 6 år; krimi; for 5 år; for 7 år; let at læse; for 4 år; for 8 år; Spillefilm; Danmark; kærlighed; ÷; for 10 år; for 9 år; for 3 år; 2000-2009; 77.7; for begynderlæsere; for 11 år; USA; for børn; for 12 år; familien; spænding; romaner; kvinder; spændende bøger; England; Sverige; Dansk; 2010-2019; opskrifter; venskab; for 13 år; for 2 år; historie; Biografier af enkelte personer; humor; Børnefilm; 77.74; sjove bøger; for folkeskolen; for 14 år; fantasy; politiromaner; erindringer; 1990-1999; rejseførere; piger; Computerspil; 79.41; 1940-1949; computerspil; drama; børn; Frilæs/0.-2. kl.; Norge; 1980-1989; 1970-1979; for 15 år; København; drenge; Billedbøger sk; Folkeskolen; Del af serie; 1960-1969; bk; unge; Læs let; amerikanske film; 1900-1999; parforhold; eventyr; veninder; Fantasy; ægteskab; dameromaner; 1950-1959; Frankrig; seriemordere; Tyskland; gå på opdagelse; Oplæsning 0.- 2. kl.; den 2. verdenskrig; far-søn forholdet; strikning; 1930-1939; forelskelse; i

Lad os udforske hvilke typer af biblioteksmaterialer der er:

In [5]:
Counter([bib.get('type') for bib in bibdata.meta]).most_common(5)

[('book', 7895),
 ('movie', 1055),
 ('audiobook', 608),
 ('game', 232),
 ('other', 119)]

Gad vide hvad `'other'` dækker over:

In [6]:
[bib['title'] for bib in bibdata.meta if bib.get('type') == 'other'][:5]

['Alt for damerne', 'Illustreret videnskab', 'Femina', 'Bo bedre', 'I form']

In [7]:
def books_by_subject(subject):
    books = [book_id for book_id in range(len(bibdata.meta)) if subject in bibdata.meta[book_id].get('subject-term', [])]
    vectors = [bibdata.genres[book] for book in books]
    mean = numpy.mean(vectors, axis=0)
    std = numpy.std(vectors, axis=0)
    weighted_books = [(numpy.linalg.norm((bibdata.genres[book_id]-mean)/std), book_id) for book_id in range(10000)]
    sorted_books = [book_id for (weight, book_id) in sorted(weighted_books)]
    return sorted_books

In [8]:
"; ".join([subject for (subject, count) in Counter(subjects).most_common(150)])

'sk; Skønlitteratur; for 6 år; krimi; for 5 år; for 7 år; let at læse; for 4 år; for 8 år; Spillefilm; Danmark; kærlighed; ÷; for 10 år; for 9 år; for 3 år; 2000-2009; 77.7; for begynderlæsere; for 11 år; USA; for børn; for 12 år; familien; spænding; romaner; kvinder; spændende bøger; England; Sverige; Dansk; 2010-2019; opskrifter; venskab; for 13 år; for 2 år; historie; Biografier af enkelte personer; humor; Børnefilm; 77.74; sjove bøger; for folkeskolen; for 14 år; fantasy; politiromaner; erindringer; 1990-1999; rejseførere; piger; Computerspil; 79.41; 1940-1949; computerspil; drama; børn; Frilæs/0.-2. kl.; Norge; 1980-1989; 1970-1979; for 15 år; København; drenge; Billedbøger sk; Folkeskolen; Del af serie; 1960-1969; bk; unge; Læs let; amerikanske film; 1900-1999; parforhold; eventyr; veninder; Fantasy; ægteskab; dameromaner; 1950-1959; Frankrig; seriemordere; Tyskland; gå på opdagelse; Oplæsning 0.- 2. kl.; den 2. verdenskrig; far-søn forholdet; strikning; 1930-1939; forelskelse; i

In [9]:
subject = 'Biografier af enkelte personer'
subject = 'computerspil'
print('\n'.join([bibdata.title_creator(book_id) for book_id in books_by_subject(subject) 
 if not subject in bibdata.meta[book_id].get('subject-term', [])][:5]))
print('---- subject:' + subject + ' ----')
print('\n'.join([bibdata.title_creator(book_id) for book_id in reversed(books_by_subject(subject))
 if subject in bibdata.meta[book_id].get('subject-term', [])][:5]))

Surf's up - Ubi Soft (game)
Spider-man 3 -  (game)
Asterix at the Olympic Games - René Goscinny (game)
Rayman - raving rabbids - Ubi Soft (game)
Brave : the search for Spirit Dancer - Mark Hughes (game)
---- subject:computerspil ----
Det 13. spil : roman - Morten Sabroe (book)
Ripper : krimi - Isabel Allende (book)
Tron - legacy - Walt Disney firma (movie)
De ukendte - Jan Solheim (book)
Minecraft : byggehåndbogen - Mojang (book)


In [10]:
bibdata.meta[9953]

{'_id': '870970-basis:50673103',
 'creator': 'Dowsett Elizabeth edt',
 'idx': 9953,
 'language': 'Dansk',
 'subject-term': ['!',
  'for 12 år',
  'for 11 år',
  'Biografier af enkelte personer',
  'for 9 år',
  'Dave Filoni',
  'Star wars - the clone wars',
  '99.4 Filoni, Dave',
  'Legetøj',
  '79.31',
  'figurer',
  'Dave',
  'for 10 år',
  'Filoni, Dave',
  'Filoni'],
 'title': 'Star wars - clone wars figurleksikon',
 'type': 'book'}

In [11]:
subject = 'let at læse'
# books = [book_id for book_id in range(len(bibdata.meta)) if subject in bibdata.meta[book_id]['subject-term']]
books = [book_id for book_id in range(len(bibdata.meta)) if subject in bibdata.meta[book_id].get('subject-term', [])]
vectors = [bibdata.genres[book] for book in books]
mean = numpy.mean(vectors, axis=0)
std = numpy.std(vectors, axis=0)

subject_books = sorted(
    [(numpy.linalg.norm((bibdata.genres[book_id]-mean)/std, ord=1), book_id) for book_id in range(10000)])
[(bibdata.title_creator(book_id), ",".join(bibdata.meta[book_id].get('subject-term',[]))) for (weight, book_id) in subject_books if not book_id in books][:10]


[('Jorden - Leonie Pratt (book)',
  'Anbefales: Basis,for 9 år,Planeter. Måner,for 8 år,÷,for 7 år,Jorden,52.43,Lette fagbøger'),
 ('Rumfart - Troels Gollander (book)',
  'Rummet,Første Fakta,Sagprosa sm,for 9 år,Faglig læsning,Tværfagligt materiale,for 8 år,for folkeskolen,62.98,Faglig letlæsning,for 7 år,Lette fagbøger,Astronautik,rumfart'),
 ('Kom frem, Bella! - Otto Dickmeiss (book)',
  'for 6-7 år,bange,sk,Skønlitteratur,for 7 år,for 6 år,hunde'),
 ('Aladdin og den magiske lampe - Katie Daynes (book)',
  'for børn,Iran,39.12,eventyr,for 9 år,iranske eventyr,sk,for 8 år,Skønlitteratur,romaner,÷,for 10 år,persiske eventyr,Eventyr, historier og fabler,Anbefales: 2. klasse'),
 ('Alle elsker Sigge - Benedikte Biørn (audiobook)',
  'heste,for 9 år,veninder,sk,for 8 år,Skønlitteratur,for 10 år,piger'),
 ('Pony & co. - Kirsten Sonne Harild (audiobook)',
  'for 11 år,heste,for 9 år,veninder,sk,Skønlitteratur,for 10 år,piger'),
 ('Gode venner og et straffespark - Christian Bieniek (book)',
